<a href="https://colab.research.google.com/github/irfankkhairullah/-spacecoffee-index.html/blob/master/Spell_Check_Indonesia_using_Pre_trained_Fasttext_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spell Check Indonesia using Pre-trained Fasttext Model

Author: Yasir Abdur Rohman <br>
GitHub: [@yasirabd](https://github.com/yasirabd)
___
Code ini terinspirasi oleh Kaggle master [@CPMPML](https://www.kaggle.com/cpmpml) yang membuat Spell Check Bahasa Inggris dengan menggunakan Word2Vec dari *pre-trained* model GoogleNews. Maka, Saya tertarik untuk membuat Spell Check Bahasa Indonesia dengan Word2Vec dari *pre-trained* model Fasttext. 

## Upgrade/Install Gensim

In [ ]:
!pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.1)


## Dowload Pre-trained Fasttext Model Indonesia
Pre-trained Fasttext model yang diunduh adalah hasil pelatihan menggunakan metode CBOW dengan *position-weight*, dalam dimensi 300, dengan panjang karakter n-gram sebesar 5, *window size* 5 dan *negatives* 10. Link kumpulan pre-trained model Fasttext dapat diunduh [disini](https://fasttext.cc/docs/en/crawl-vectors.html).

In [ ]:
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz

--2020-03-03 09:09:04--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4507049071 (4.2G) [application/octet-stream]
Saving to: ‘cc.id.300.bin.gz’

cc.id.300.bin.gz    100%[===================>]   4.20G  22.4MB/s    in 3m 23s  

2020-03-03 09:12:28 (21.1 MB/s) - ‘cc.id.300.bin.gz’ saved [4507049071/4507049071]



Extract model dengan menggunakan `gunzip`

In [ ]:
! gunzip cc.id.300.bin.gz

## Import Libraries

In [ ]:
import io
import time
from datetime import timedelta
import logging
logging.basicConfig(level=logging.INFO)
import gensim
from gensim.models.wrappers import FastText

## Load Fasttext Model using Gensim
Saat menjalankan fungsi `load_fasttext_format` membutuhkan RAM yang besar. Dengan default RAM Google Colab sebesar 12 GB masih kurang, maka **harus** upgrade RAM hingga 25 GB dengan cara pilih *high-RAM runtime*.

In [ ]:
model = FastText.load_fasttext_format('cc.id.300.bin')

INFO:gensim.models.deprecated.fasttext_wrapper:loading 2000000 words for fastText model from cc.id.300.bin
INFO:gensim.models.deprecated.fasttext_wrapper:loading weights for 2000000 words for fastText model from cc.id.300.bin
INFO:gensim.models.deprecated.fasttext_wrapper:loaded (2000000, 300) weight matrix for fastText model from cc.id.300.bin


Pada pre-trained model Fasttext terdapat 2.000.000 (dua juta) *vocabulary*

In [ ]:
# check vocab pada pre-trained model
vocab = model.wv.vocab
len(vocab)

2000000

In [ ]:
# lihat 10 kata/karakter paling awal pada vocab
list(model.wv.vocab)[:10]

[',', '.', '</s>', 'yang', 'dan', '"', 'di', ')', '(', 'dengan']

## Create Index for each word in Vocabulary
Buat *dictionary word-rank* pada tiap kata/karakter yang terdapat pada *vocabulary*. 

In [ ]:
words = list(model.wv.vocab)

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

In [ ]:
import itertools

# cetak 10 word-rank teratas
dict(itertools.islice(WORDS.items(), 10))

{'"': 5,
 '(': 8,
 ')': 7,
 ',': 0,
 '.': 1,
 '</s>': 2,
 'dan': 4,
 'dengan': 9,
 'di': 6,
 'yang': 3}

## Peter Norvig Spelling Corrector
Salah satu metode paling sederhana untuk Spelling Corrector adalah dengan metode yang diterapkan oleh [Peter Norvig.](https://norvig.com/spell-correct.html)

### Perbedaan
Terdapat perbedaaan pada code Peter Norvig dengan code CPMP untuk penerapan Spell Check. Perbedaannya jika menggunakan **code Peter Norvig dengan menghitung frekuensi dari tiap kata dari kamus**, sedangkan **code CPMP dengan menggunakan ranking dari Word2Vec model**.

### Cara Kerja
Spell Check ini menggunakan Teorema Bayes untuk menemukan *correction c* dengan memilih *probability* terbesar dari semua *probability candidate correction*. Metode tersebut dapat dijabarkan menjadi 4 bagian:
1. **Selection Mechanism**: `argmax`, dengan memilih *candidate* yang memiliki *probability* terbesar.
2. **Candidate Model:** `c ∈ candidates`, didapatkan *candidate corrections c*, untuk dipertimbangkan.
3. **Languange Model**: `P(c)`, *probability* dari kemunculan *candidate corretion c* pada kamus. Pada penerapannya menggunakan kamus Word2Vec model.
4. **Error Model**: `P(w|c)`, *probability* apabila yang diketik adalah kata *w* sedangkan yang dimaksud adalah *c*. Sebagai contoh, *probability* `P(mkan|makan)` lebih tinggi dibandingkan dengan *probability* `P(mkanxxxyz|makan)` yang lebih rendah.

In [ ]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

## Spelling Test

In [ ]:
correction('kcingg')

'kucing'

In [ ]:
correction('mnyedihknn')

'menyedihkan'

In [ ]:
correction('mrdeka ')

'merdeka'

In [ ]:
correction('indonesa')

'indonesa'

In [ ]:
# kata 'indonesa' terdapat pada vocab
'indonesa' in model.wv.vocab

True

In [ ]:
correction('jakata')

'jakata'

In [ ]:
# kata 'jakata' terdapat pada vocab
'jakata' in model.wv.vocab

True

In [ ]:
correction('J4karta')

'Jakarta'

## Future Improvement
- Masih banyak terdapat kata-kata yang tidak baku pada *word-vector* yang terdapat pada *pre-trained* model. Oleh karena itu, dibutuhkan validasi pada KBBI.
- Pembuatan `testset` untuk Spell Check Bahasa Indonesia dikarenakan sampai saat ini *author* masih belum menemukan standar `testset` untuk *spelling corrector* Bahasa Indonesia.